# SNN Weight & Threshold Balancing - Convolutional

In [1]:
import torch
from torchvision.transforms import ToTensor#, Compose, Normalize
from torchvision.datasets import MNIST

import snntorch as snn
import snntorch.functional as SF

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
# data alread 0-1 normalised, simply convert to tensor
transform_data = ToTensor()

# Load the data
batch_size = 100
train_dataset = MNIST(root = './mnist/', train = True, download = True, transform=transform_data)
test_dataset = MNIST(root = './mnist/', train = False, download = True, transform=transform_data)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

## Convolutional ReLU Neural Network

In [6]:
# convolutional neural network
class Conv_Net(torch.nn.Module):
    def __init__(self, n_x: int, n_h: list, n_y: list, kernel_size: int=5):
        super().__init__()

        self.in_layer = torch.nn.Conv2d(n_x, n_h[0], kernel_size, bias=False)
        self.h1_layer = torch.nn.Conv2d(n_h[0], n_h[1], kernel_size, bias=False)
        self.h2_layer = torch.nn.Linear(n_y[0], n_y[1], bias=False)
        self.pooling = torch.nn.AvgPool2d(2)
        self.dropout = torch.nn.Dropout()
        self.activator = torch.nn.ReLU()

    def forward(self, x):
        # Flatten images
        # x = x.view(x.size(0), -1)
        
        inp = self.dropout(self.activator(self.pooling(self.in_layer(x))))
        h1 = self.dropout(self.activator(self.pooling(self.h1_layer(inp))))
        
        # vectorise image
        h1 = h1.view(h1.size(0), -1)
        y = self.activator(self.h2_layer(h1))

        return y
    
    def save_parameters(self, path: str):
        torch.save(self.in_layer, path + "0.pt")
        torch.save(self.h1_layer, path + "1.pt")
        torch.save(self.h2_layer, path + "2.pt")

    def load_parameters(self, path: str):
        self.in_layer = torch.load(path + "0.pt", weights_only=False)
        self.h1_layer = torch.load(path + "1.pt", weights_only=False)
        self.h2_layer = torch.load(path + "2.pt", weights_only=False)

In [9]:
def init_weights(m):
    if isinstance(m, torch.nn.Linear) or isinstance(m, torch.nn.Conv2d):
        torch.nn.init.uniform_(m.weight, -0.1, 0.1)

conv_net = Conv_Net(1, [12, 64], [1024, 10]).to(device)
conv_net.apply(init_weights)
optimiser = torch.optim.Adam(conv_net.parameters())

# optimiser used in the original paper seems to kill the gradients, so we're just going to use adam
# optimiser = torch.optim.SGD(conv_net.parameters(), lr=.01, momentum=0.5)

conv_net.train()

Conv_Net(
  (in_layer): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1), bias=False)
  (h1_layer): Conv2d(12, 64, kernel_size=(5, 5), stride=(1, 1), bias=False)
  (h2_layer): Linear(in_features=1024, out_features=10, bias=False)
  (pooling): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (dropout): Dropout(p=0.5, inplace=False)
  (activator): ReLU()
)

In [10]:
# Training model
num_epochs = 15
for epoch in range(num_epochs):
    # Go trough all samples in train dataset
    for i, (images, labels) in enumerate(train_loader):
        # Get from dataloader and send to device
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = conv_net(images)
        # Compute loss
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        # Backward and optimize
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        # Display
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/15], Step [100/600], Loss: 0.7449
Epoch [1/15], Step [200/600], Loss: 0.5508
Epoch [1/15], Step [300/600], Loss: 0.3478
Epoch [1/15], Step [400/600], Loss: 0.3962
Epoch [1/15], Step [500/600], Loss: 0.3962
Epoch [1/15], Step [600/600], Loss: 0.1065
Epoch [2/15], Step [100/600], Loss: 0.3321
Epoch [2/15], Step [200/600], Loss: 0.1846
Epoch [2/15], Step [300/600], Loss: 0.2293
Epoch [2/15], Step [400/600], Loss: 0.2776
Epoch [2/15], Step [500/600], Loss: 0.1158
Epoch [2/15], Step [600/600], Loss: 0.1491
Epoch [3/15], Step [100/600], Loss: 0.1828
Epoch [3/15], Step [200/600], Loss: 0.1167
Epoch [3/15], Step [300/600], Loss: 0.0789
Epoch [3/15], Step [400/600], Loss: 0.1116
Epoch [3/15], Step [500/600], Loss: 0.1525
Epoch [3/15], Step [600/600], Loss: 0.1324
Epoch [4/15], Step [100/600], Loss: 0.1401
Epoch [4/15], Step [200/600], Loss: 0.0405
Epoch [4/15], Step [300/600], Loss: 0.1163
Epoch [4/15], Step [400/600], Loss: 0.2690
Epoch [4/15], Step [500/600], Loss: 0.1170
Epoch [4/15

In [11]:
# Evaluate model accuracy on test after training
# Set model in eval mode!
conv_net.eval()
# Evaluate
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        # Get images and labels from test loader
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass and predict class using max
        outputs = conv_net(images)
        _, predicted = torch.max(outputs.data, 1)
        # Check if predicted class matches label
        # and count number of correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
# Compute final accuracy and display
accuracy = correct/total
print(f'Evaluation after training, test accuracy: {accuracy:.4f}')

Evaluation after training, test accuracy: 0.9894


In [12]:
conv_net.save_parameters("params/conv")